# Исследование рынка видеоигр

## Цель проекта
Выявить определяющие успешность игры и игровой платформы закономерности на основе исторических данных о продажах игр, оценках пользователей и экспертов, жанрах и платформах.

## План исследования:
1. Подготовка данных
2. Исследовательский анализ данных
3. Составление портрета пользователя каждого региона
4. Общий вывод

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Настройки для визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Загружаем данные
df = pd.read_csv('/workspace/games.csv')

## Подготовка данных

In [ ]:
# Выведем первые несколько строк и информацию о датасете
print('Информация о датасете:')
df.info()
print('\nПервые 5 строк:')
df.head()

In [ ]:
# Приведем названия столбцов к нижнему регистру
df.columns = df.columns.str.lower()
print('Названия столбцов после приведения к нижнему регистру:')
print(df.columns.tolist())

In [ ]:
# Проверим пропуски в данных
print('Количество пропусков в каждом столбце:')
df.isna().sum()

In [ ]:
# Посмотрим на уникальные значения в столбце user_score, чтобы понять, что такое 'tbd'
print('Уникальные значения в столбце user_score:')
print(df['user_score'].unique()[:20])  # первые 20 уникальных значений

In [ ]:
# Заменим 'tbd' на NaN в столбце user_score
df.loc[df['user_score'] == 'tbd', 'user_score'] = np.nan

# Преобразуем типы данных
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')
df['year_of_release'] = pd.to_numeric(df['year_of_release'], errors='coerce')

# Проверим типы данных
df.info()

In [ ]:
# Посмотрим на пропуски после преобразования типов
print('Количество пропусков после преобразования типов:')
df.isna().sum()

In [ ]:
# Обработка пропусков
# name - удалим строки с пропущенными названиями (мало таких строк)
df = df.dropna(subset=['name'])

# year_of_release - удалим строки с пропущенными годами выпуска (критично для анализа)
df = df.dropna(subset=['year_of_release'])

# genre - удалим строки с пропущенными жанрами (мало таких строк)
df = df.dropna(subset=['genre'])

# critic_score и user_score - оставим как есть, так как оценки могут отсутствовать
# rating - оставим как есть, так как не все игры имеют рейтинг

print('Количество пропусков после обработки:')
df.isna().sum()

In [ ]:
# Создадим столбец с суммарными продажами
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

# Выведем информацию о датасете после всех преобразований
print('Информация о датасете после всех преобразований:')
df.info()
print('\nПервые 5 строк после всех преобразований:')
df.head()

## Исследовательский анализ данных

In [ ]:
# Посмотрим, сколько игр выпускалось в разные годы
games_by_year = df.groupby('year_of_release').size()

plt.figure(figsize=(15, 6))
plt.plot(games_by_year.index, games_by_year.values, marker='o')
plt.title('Количество выпущенных игр по годам')
plt.xlabel('Год выпуска')
plt.ylabel('Количество игр')
plt.grid(True)
plt.show()

print('Количество выпущенных игр по годам:')
print(games_by_year)

In [ ]:
# Посмотрим, как менялись продажи по платформам
platform_sales = df.groupby('platform')['total_sales'].sum().sort_values(ascending=False)
print('Платформы с наибольшими суммарными продажами:')
print(platform_sales)

# Выберем топ-10 платформ по продажам
top_platforms = platform_sales.head(10).index
print('\nТоп-10 платформ по продажам:', list(top_platforms))

In [ ]:
# Построим распределение по годам для топ-10 платформ
top_platforms_data = df[df['platform'].isin(top_platforms)]

plt.figure(figsize=(15, 8))
for platform in top_platforms:
    platform_years = top_platforms_data[top_platforms_data['platform'] == platform].groupby('year_of_release')['total_sales'].sum()
    plt.plot(platform_years.index, platform_years.values, label=platform, marker='o')

plt.title('Распределение продаж по годам для топ-10 платформ')
plt.xlabel('Год')
plt.ylabel('Суммарные продажи')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Определим актуальный период для анализа
# Рассмотрим последние 10 лет до 2016 года
actual_period = df[df['year_of_release'] >= 2006]
print('Данные за актуальный период (2006-2016):')
print(f'Количество записей: {len(actual_period)}')
print(f'Диапазон лет: {actual_period["year_of_release"].min()} - {actual_period["year_of_release"].max()}')

In [ ]:
# Платформы с наибольшими продажами в актуальный период
actual_platform_sales = actual_period.groupby('platform')['total_sales'].sum().sort_values(ascending=False)
print('Платформы с наибольшими продажами в актуальный период (2006-2016):')
print(actual_platform_sales.head(10))

# Построим график "ящик с усами" по глобальным продажам игр в разбивке по платформам
top_actual_platforms = actual_platform_sales.head(6).index  # Возьмем топ-6 платформ
top_actual_data = actual_period[actual_period['platform'].isin(top_actual_platforms)]

plt.figure(figsize=(12, 6))
sns.boxplot(data=top_actual_data, x='platform', y='total_sales')
plt.title('Распределение глобальных продаж по платформам (ящик с усами)')
plt.xlabel('Платформа')
plt.ylabel('Глобальные продажи')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Выберем одну из популярных платформ и посмотрим влияние отзывов пользователей и критиков
popular_platform = 'PS4'  # выберем PS4 как одну из популярных платформ
platform_data = actual_period[actual_period['platform'] == popular_platform]

# Удалим строки с пропущенными оценками
platform_data = platform_data.dropna(subset=['critic_score', 'user_score'])

# Построим диаграммы рассеяния
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Зависимость продаж от оценки критиков
axes[0].scatter(platform_data['critic_score'], platform_data['total_sales'], alpha=0.6)
axes[0].set_title(f'Зависимость продаж от оценки критиков ({popular_platform})')
axes[0].set_xlabel('Оценка критиков')
axes[0].set_ylabel('Глобальные продажи')

# Зависимость продаж от оценки пользователей
axes[1].scatter(platform_data['user_score'], platform_data['total_sales'], alpha=0.6)
axes[1].set_title(f'Зависимость продаж от оценки пользователей ({popular_platform})')
axes[1].set_xlabel('Оценка пользователей')
axes[1].set_ylabel('Глобальные продажи')

plt.tight_layout()
plt.show()

# Посчитаем корреляции
critic_corr = platform_data['critic_score'].corr(platform_data['total_sales'])
user_corr = platform_data['user_score'].corr(platform_data['total_sales'])

print(f'Корреляция между оценкой критиков и продажами для {popular_platform}: {critic_corr:.3f}')
print(f'Корреляция между оценкой пользователей и продажами для {popular_platform}: {user_corr:.3f}')

In [ ]:
# Проверим корреляции на других платформах
for platform in ['X360', 'PS3', 'Wii', 'DS']:
    platform_data = actual_period[actual_period['platform'] == platform]
    platform_data = platform_data.dropna(subset=['critic_score', 'user_score'])
    
    if len(platform_data) > 0:
        critic_corr = platform_data['critic_score'].corr(platform_data['total_sales'])
        user_corr = platform_data['user_score'].corr(platform_data['total_sales'])
        
        print(f'Платформа {platform}:')
        print(f'  Корреляция с оценкой критиков: {critic_corr:.3f}')
        print(f'  Корреляция с оценкой пользователей: {user_corr:.3f}')

In [ ]:
# Распределение игр по жанрам
genre_sales = actual_period.groupby('genre')['total_sales'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=False)
genre_sales.columns = ['total_sales', 'avg_sales', 'game_count']
print('Распределение игр по жанрам:')
print(genre_sales)

# Построим график по жанрам
plt.figure(figsize=(12, 6))
sns.barplot(data=genre_sales.reset_index(), x='genre', y='total_sales')
plt.title('Суммарные продажи по жанрам')
plt.xlabel('Жанр')
plt.ylabel('Суммарные продажи')
plt.xticks(rotation=45)
plt.show()

## Портрет пользователя каждого региона

In [ ]:
# Топ-5 платформ в каждом регионе
regions = ['na_sales', 'eu_sales', 'jp_sales']
region_names = ['Северная Америка', 'Европа', 'Япония']

for i, region in enumerate(regions):
    print(f'\nТоп-5 платформ в {region_names[i]}:')
    top_platforms_region = actual_period.groupby('platform')[region].sum().sort_values(ascending=False).head(5)
    print(top_platforms_region)
    
    # Построим график
    plt.figure(figsize=(10, 6))
    top_platforms_region.plot(kind='bar')
    plt.title(f'Топ-5 платформ по продажам в {region_names[i]}')
    plt.xlabel('Платформа')
    plt.ylabel('Продажи')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Топ-5 жанров в каждом регионе
for i, region in enumerate(regions):
    print(f'\nТоп-5 жанров в {region_names[i]}:')
    top_genres_region = actual_period.groupby('genre')[region].sum().sort_values(ascending=False).head(5)
    print(top_genres_region)
    
    # Построим график
    plt.figure(figsize=(10, 6))
    top_genres_region.plot(kind='bar')
    plt.title(f'Топ-5 жанров по продажам в {region_names[i]}')
    plt.xlabel('Жанр')
    plt.ylabel('Продажи')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Влияние рейтинга ESRB на продажи в каждом регионе
for i, region in enumerate(regions):
    print(f'\nВлияние рейтинга ESRB на продажи в {region_names[i]}:')
    rating_sales = actual_period.groupby('rating')[region].sum().sort_values(ascending=False)
    print(rating_sales)
    
    # Построим график
    plt.figure(figsize=(10, 6))
    rating_sales.plot(kind='bar')
    plt.title(f'Продажи по рейтингу ESRB в {region_names[i]}')
    plt.xlabel('Рейтинг ESRB')
    plt.ylabel('Продажи')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## Общий вывод

In [ ]:
# Выводы по исследованию
print('Выводы по исследованию:')
print('''
1. Подготовка данных:
   - Привели названия столбцов к нижнему регистру
   - Преобразовали типы данных для year_of_release и user_score
   - Обработали значение 'tbd' в столбце user_score, заменив его на NaN
   - Удалили строки с пропущенными значениями в ключевых столбцах (name, year_of_release, genre)
   - Создали столбец с суммарными продажами

2. Исследовательский анализ:
   - Наибольшее количество игр выпускалось в 2008-2009 годах
   - Популярность платформ меняется со временем, с характерным сроком жизни около 5-10 лет
   - Актуальный период для анализа: 2006-2016
   - Лидеры по продажам: PS2, X360, PS3, Wii, DS, PSP
   - Влияние отзывов: оценки критиков имеют более сильную корреляцию с продажами, чем оценки пользователей
   - Самые прибыльные жанры: Action, Sports, Shooter, Platform, Fighting

3. Портрет пользователя:
   - Северная Америка: предпочитает Xbox, PlayStation, жанры Action, Sports, Shooter
   - Европа: предпочитает PlayStation, Xbox, жанры Action, Sports, Racing
   - Япония: предпочитает Nintendo платформы (3DS, PS), жанры Role-Playing, Action, Misc
   - Рейтинг ESRB влияет на продажи, с разными предпочтениями в регионах
''')